In [1]:
import os
from scipy.io import loadmat
import numpy as np
import makefilter
from scipy.signal import sosfiltfilt, hilbert
from scipy.signal import savgol_filter
import pickle
from itertools import combinations

In [2]:

def list_data_folders(directory_path,pattern):
    """
    Lists all folders in the given directory that start with pattern.
    
    Parameters:
    - directory_path: A string representing the path to the directory to search in.

    - pattern: A string representing the pattern at the beginning of the folders' name.
    
    Returns:
    - A list of folder names that meet the criteria.
    """
    # Ensure the directory exists
    if not os.path.exists(directory_path):
        print(f"The directory {directory_path} does not exist.")
        return []

    # Get all items in the directory
    all_items = os.listdir(directory_path)

    # Filter for directories that start with pattern
    folders_starting_with_pattern = [item for item in all_items
                                if os.path.isdir(os.path.join(directory_path, item)) and item.startswith(pattern)]

    return folders_starting_with_pattern

def list_files_in_directory(directory_path):
    """
    Lists all the files in the specified directory.
    
    Parameters:
    - directory_path: A string representing the path to the directory to search in.
    
    Returns:
    - A list of file names contained in the directory.
    """
    # Ensure the directory exists
    if not os.path.exists(directory_path):
        print(f"The directory {directory_path} does not exist.")
        return []

    # Get all items in the directory
    all_items = os.listdir(directory_path)

    # Filter out only files
    files_only = [item for item in all_items if os.path.isfile(os.path.join(directory_path, item))]

    return files_only

def open_mat_file(file_path):
    """
    Opens a .mat file and returns its contents.
    
    Parameters:
    - file_path: A string representing the path to the .mat file.
    
    Returns:
    - A dictionary containing variables loaded from the .mat file.
    """
    # Load the .mat file
    data = loadmat(file_path)
    
    return data

def segment_time_series(data, segment_duration, overlap_duration, sampling_rate):
    """
    Segments each time series in a 2D NumPy array into smaller time series segments 
    based on specified duration, overlap, and sampling rate.

    Parameters:
    - data (numpy.ndarray): A 2D NumPy array containing the time series data. Each 
    column represents a single time series.
    - segment_duration (float): The duration of each segment in seconds. Determines
     the length of each segment generated.
    - overlap_duration (float): The duration of overlap between consecutive segments
     in seconds. This specifies how much each segment should overlap with the next.
    - sampling_rate (int): The number of samples per second in the time series data.
     This is used to calculate the number of samples per segment and the overlap in 
     samples.

    Returns:
    - numpy.ndarray: A 3D NumPy array where the 1st dimension represents a segmented portion 
    of the original time series. The 2nd dimension in this array equals the number 
    of samples per segment, determined by the `segment_duration` and `sampling_rate`.
    """
    # Calculate parameters
    samples_per_segment = int(segment_duration * sampling_rate)
    overlap_samples = int(overlap_duration * sampling_rate)
    step_size = samples_per_segment - overlap_samples
    
    segments = []
    len_data,_ = data.shape
    for start in range(0, len_data - samples_per_segment + 1, step_size):
        segment = data[start:start+samples_per_segment,:]
        segments.append(segment)
            
    return np.array(segments)

def save_to_pickle(dict_obj, file_path):
    """
    Saves a given dictionary to a pickle file.

    Parameters:
    - dict_obj (dict): The dictionary to be saved.
    - file_path (str): The path to the file where the dictionary will be saved.

    Returns:
    - None
    """
    with open(file_path, 'wb') as file:
        # Serialize the dictionary and save it to the file
        pickle.dump(dict_obj, file)
    print(f"Data successfully saved to {file_path}.")

def corr_matrix_stack(corr_matrix_dic):
    """
    Stacks correlation matrices along axis 0 and tracks their sizes and identifiers.

    This function takes a dictionary of correlation matrices (2D NumPy arrays) and
    stacks these matrices vertically. It also compiles a list of the sizes of these
    matrices along axis 0, alongside their identifiers.

    Parameters:
    - corr_matrix_dic (dict): A dictionary where the keys are identifiers (e.g., file names)
      and the values are correlation matrices (2D NumPy arrays). Each matrix is assumed to
      have the same number of columns but can vary in the number of rows.

    Returns:
    - numpy.ndarray: A single 2D NumPy array resulting from stacking all the input matrices
      along axis 0.
    - list of tuples: Each tuple contains an identifier (key from the input dictionary) and
      an integer representing the size (number of rows) of the corresponding matrix before
      stacking. This list maintains the order in which matrices were stacked.
    """
    
    # Initialize a list to keep track of each matrix's identifier and its number of rows
    size_matrices = []
    
    # Initialize a list to hold all matrices for concatenation
    matrix_list = []
    
    # Iterate over the dictionary items
    for file, matrix in corr_matrix_dic.items():
        # Append each matrix to the list for later concatenation
        matrix_list.append(matrix)
        # Append a tuple of the matrix's identifier and its number of rows to the tracking list
        size_matrices.append((file, matrix.shape[0]))
    
    # Concatenate all matrices vertically
    stacked_array = np.concatenate(matrix_list, axis=0)
    
    # Return the stacked array and the list of identifiers with their corresponding matrix sizes
    return stacked_array, size_matrices

def corr_dist(A,B):
    from scipy.linalg import norm
    D = np.transpose(np.conj(A))@B
    dist = np.real(np.log(1.0/(np.trace(D)/(norm(A)*norm(B)))))
    return dist

def sort_key(filename):
    trial_part = int(filename.split('_')[-1].split('.')[0])  # Extract the trial number
    return trial_part


In [3]:
def segment_time_series_taplocked(data, segment_duration, sampling_rate, tr_samples):
    """
    Segments each time series in a 2D NumPy array into smaller time series segments 
    based on specified duration, and sampling rate.

    Parameters:
    - data (numpy.ndarray): A 2D NumPy array containing the time series data. Each 
    column represents a single time series.
    - segment_duration (float): The duration of each segment in seconds. Determines
     the length of each segment generated.
    - sampling_rate (int): The number of samples per second in the time series data.
     This is used to calculate the number of samples per segment and the overlap in 
     samples.
    - tr_samples: index of taps in the samples of a single trial

    Returns:
    - numpy.ndarray: A 3D NumPy array where the 1st dimension represents a segmented portion 
    of the original time series. The 2nd dimension in this array equals the number 
    of samples per segment, determined by the `segment_duration` and `sampling_rate`.
    """
    # Calculate parameters
    samples_per_segment = int(segment_duration * sampling_rate)

    segments = []
    for start in tr_samples:
        segment = data[start-samples_per_segment:start,:]
        segments.append(segment)

    return np.array(segments)

In [5]:
# directory_path = '/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/'  # Replace this with the path to your directory
# folders = list_data_folders(directory_path,'20')

#Butterworth filters.
butt_filter1,butt_w,butt_h = makefilter.makefiltersos(2000,50,60)
# window_len = 2.0
window_len = 0.5
# overlap = 0.5
# overlap_len = overlap*window_len
butt_filter2,butt_w,butt_h = makefilter.makefiltersos(2000,1.0/window_len,0.5/window_len)

In [4]:
folders = ['20220713','20220721',
           '20220804','20220808',
           '20220810','20220811',
           '20220815','20220816',
           '20221003','2022100401',
           '2022100402','20221005']

In [8]:
folder=folders[0]

In [9]:
print(folder)

20220713


In [17]:
files_dic = {}

In [9]:
directory_path = '/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/' + folder

In [11]:
directory_path

'/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/20220713'

In [13]:
files = list_files_in_directory(directory_path)

In [14]:
files

['subj1_tr_8.mat',
 'subj1_tr_4.mat',
 'subj2_tr_4.mat',
 'subj2_tr_9.mat',
 'subj1_tr_11.mat',
 'subj1_tr_5.mat',
 'subj1_tr_1.mat',
 'subj1_tr_10.mat',
 'subj1_tr_3.mat',
 'subj2_tr_2.mat',
 'subj2_tr_1.mat',
 'subj1_tr_6.mat',
 'subj1_tr_9.mat',
 'subj2_tr_6.mat',
 'subj2_tr_3.mat',
 'subj2_tr_12.mat',
 'subj1_tr_7.mat',
 'subj2_tr_8.mat',
 'subj2_tr_11.mat',
 'subj1_tr_2.mat',
 'subj2_tr_7.mat',
 'subj2_tr_5.mat',
 'subj1_tr_12.mat',
 'subj2_tr_10.mat']

In [11]:
# load event time points
beha_filename = '/home/zhibinz2/Documents/GitHub/finger_tapping_behavioral_data/clean_' + str(folder) + '_bpchan.mat'
beh_data =  loadmat(beha_filename)
samples = beh_data['samples'][0]

In [45]:
# Compute difference(error) of the samples
tr=0
tr_samples = samples[tr]
num_tp=len(tr_samples)
tr_error=np.zeros(num_tp)
for tp in range(num_tp):
    tr_error[tp]=np.abs(tr_samples[tp][0]-tr_samples[tp][1])

In [18]:
for filename in files:
    subject, rest = filename.split('_', 1)
    if subject not in files_dic:
        files_dic[subject] = {}
    file = folder + '/' + filename
    files_dic[subject][filename] = directory_path + '/' + filename
print('files listed.')

files listed.


In [19]:
files_dic['subj1'].keys()

dict_keys(['subj1_tr_8.mat', 'subj1_tr_4.mat', 'subj1_tr_11.mat', 'subj1_tr_5.mat', 'subj1_tr_1.mat', 'subj1_tr_10.mat', 'subj1_tr_3.mat', 'subj1_tr_6.mat', 'subj1_tr_9.mat', 'subj1_tr_7.mat', 'subj1_tr_2.mat', 'subj1_tr_12.mat'])

In [20]:
files_dic['subj2'].keys()

dict_keys(['subj2_tr_4.mat', 'subj2_tr_9.mat', 'subj2_tr_2.mat', 'subj2_tr_1.mat', 'subj2_tr_6.mat', 'subj2_tr_3.mat', 'subj2_tr_12.mat', 'subj2_tr_8.mat', 'subj2_tr_11.mat', 'subj2_tr_7.mat', 'subj2_tr_5.mat', 'subj2_tr_10.mat'])

In [26]:
subject_matrices = {}

subject 1

In [27]:
subject='subj1'
subj=0

In [28]:
print(subject) 

subj1


In [29]:
corr_matrices = {}
subject_files = list(files_dic[subject].keys())
subject_files = sorted(subject_files,key=sort_key)

In [30]:
subject_files

['subj1_tr_1.mat',
 'subj1_tr_2.mat',
 'subj1_tr_3.mat',
 'subj1_tr_4.mat',
 'subj1_tr_5.mat',
 'subj1_tr_6.mat',
 'subj1_tr_7.mat',
 'subj1_tr_8.mat',
 'subj1_tr_9.mat',
 'subj1_tr_10.mat',
 'subj1_tr_11.mat',
 'subj1_tr_12.mat']

In [31]:
tr=1
file='subj1_tr_1.mat'
path = files_dic[subject][file]
filename = folder + '/' + file
filename

'20220713/subj1_tr_1.mat'

In [32]:
path

'/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/20220713/subj1_tr_1.mat'

In [33]:
#Import the data from a sample file.
data = open_mat_file(path)
#Get signal, filter and downsample.
signal = data['agr_source_data']
filtered_signal = sosfiltfilt(butt_filter1, signal, axis=0)
filtered_signal = sosfiltfilt(butt_filter2, filtered_signal, axis=0)
# downsampled_signal = filtered_signal[::10,:]
# analytic_signal = hilbert(downsampled_signal,axis=0)
# no downsampling
analytic_signal = hilbert(filtered_signal,axis=0)
sg = int(np.floor(100/(25))*2+1)
ts1 = savgol_filter(np.real(analytic_signal),sg,1,axis = 0,mode = 'interp')
ts2 = savgol_filter(np.imag(analytic_signal),sg,1,axis = 0,mode = 'interp')
ts3 = ts1+1j*ts2


In [46]:
# time_windows = segment_time_series(ts3, window_len, overlap_len, 200)
# segment data with tap-locked windows
tr_samples_subj = samples[tr][:,subj]
time_windows = segment_time_series_taplocked(ts3, window_len, 2000, tr_samples_subj)
# 50s

In [51]:
np.shape(time_windows[0])

(1000, 448)

In [52]:
corr_matrix = []
for window in time_windows:
    corr_matrix.append(np.corrcoef(window, rowvar=False))
corr_matrix = np.array(corr_matrix)
corr_matrix = corr_matrix - np.mean(corr_matrix,axis=0)
corr_matrices[filename] = corr_matrix

In [53]:
for tr, file in enumerate(subject_files):
    print(tr)
    print(file)
    path = files_dic[subject][file]
    filename = folder + '/' + file
    # print(path)
    #Import the data from a sample file.
    data = open_mat_file(path)
    #Get signal, filter and downsample.
    signal = data['agr_source_data']
    filtered_signal = sosfiltfilt(butt_filter1, signal, axis=0)
    filtered_signal = sosfiltfilt(butt_filter2, filtered_signal, axis=0)
    # downsampled_signal = filtered_signal[::10,:]
    # analytic_signal = hilbert(downsampled_signal,axis=0)
    # no downsampling
    analytic_signal = hilbert(filtered_signal,axis=0)
    sg = int(np.floor(100/(25))*2+1)
    ts1 = savgol_filter(np.real(analytic_signal),sg,1,axis = 0,mode = 'interp')
    ts2 = savgol_filter(np.imag(analytic_signal),sg,1,axis = 0,mode = 'interp')
    ts3 = ts1+1j*ts2
    
    # time_windows = segment_time_series(ts3, window_len, overlap_len, 200)
    # segment data with tap-locked windows
    tr_samples = samples[tr][:,subj]
    time_windows = segment_time_series_taplocked(ts3, window_len, 2000, tr_samples)

    corr_matrix = []
    for window in time_windows:
        corr_matrix.append(np.corrcoef(window, rowvar=False))
    corr_matrix = np.array(corr_matrix)
    corr_matrix = corr_matrix - np.mean(corr_matrix,axis=0)
    corr_matrices[filename] = corr_matrix
print('correlation matrices calculated.')
# 9 min for 12 files in one subject
# 9 x 12 files x 2 subjects x 12 session = 43 hours

0
subj1_tr_1.mat
1
subj1_tr_2.mat
2
subj1_tr_3.mat
3
subj1_tr_4.mat
4
subj1_tr_5.mat
5
subj1_tr_6.mat
6
subj1_tr_7.mat
7
subj1_tr_8.mat
8
subj1_tr_9.mat
9
subj1_tr_10.mat
10
subj1_tr_11.mat
11
subj1_tr_12.mat
correlation matrices calculated.


In [ ]:
matrix,sizes = corr_matrix_stack(corr_matrices)

In [29]:
corr_matrices.keys()

dict_keys(['20220713/subj1_tr_1.mat', '20220713/subj1_tr_2.mat', '20220713/subj1_tr_3.mat', '20220713/subj1_tr_4.mat', '20220713/subj1_tr_5.mat', '20220713/subj1_tr_6.mat', '20220713/subj1_tr_7.mat', '20220713/subj1_tr_8.mat', '20220713/subj1_tr_9.mat', '20220713/subj1_tr_10.mat', '20220713/subj1_tr_11.mat', '20220713/subj1_tr_12.mat'])

In [27]:
sizes

[('20220713/subj1_tr_1.mat', 128),
 ('20220713/subj1_tr_2.mat', 148),
 ('20220713/subj1_tr_3.mat', 124),
 ('20220713/subj1_tr_4.mat', 116),
 ('20220713/subj1_tr_5.mat', 145),
 ('20220713/subj1_tr_6.mat', 146),
 ('20220713/subj1_tr_7.mat', 114),
 ('20220713/subj1_tr_8.mat', 141),
 ('20220713/subj1_tr_9.mat', 132),
 ('20220713/subj1_tr_10.mat', 140),
 ('20220713/subj1_tr_11.mat', 145),
 ('20220713/subj1_tr_12.mat', 147)]

In [28]:
np.shape(matrix)

(1626, 448, 448)

In [30]:
len(matrix)

1626

In [32]:
dist_matrix = np.zeros((len(matrix),len(matrix)))


In [ ]:
for i,j in combinations(range(len(matrix)),2):
    print(i)
    matrix1 = matrix[i,:,:]
    matrix2 = matrix[j,:,:]
    dist_matrix[i,j] = corr_dist(matrix1,matrix2)
    dist_matrix[j,i] = dist_matrix[i,j]
        
print('distance matrices calculated.')

In [ ]:
subject_matrices[subject] = {'distances': dist_matrix, 'sizes':sizes}

In [ ]:
subject_matrices.keys()

In [ ]:
subject_matrices['subj1'].keys()

In [ ]:
np.shape(subject_matrices['subj1']['distances'])

In [ ]:
subject_matrices['subj1']['sizes']

In [6]:
for folder in folders:
    print(folder)
    files_dic = {}
#    for folder in folders:
#    folder_dic = {}
    directory_path = '/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/' + folder
    files = list_files_in_directory(directory_path)

    # load event time points
    beha_filename = '/home/zhibinz2/Documents/GitHub/finger_tapping_behavioral_data/clean_' + str(folder) + '_bpchan.mat'
    beh_data =  loadmat(beha_filename)
    samples = beh_data['samples'][0]

    for filename in files:
        subject, rest = filename.split('_', 1)
        if subject not in files_dic:
            files_dic[subject] = {}
        file = folder + '/' + filename
        files_dic[subject][filename] = directory_path + '/' + filename
    print('files listed.')

    subject_matrices = {}
    for subj,subject in enumerate(['subj1','subj2']):
        print(subj)
        print(subject) 
        corr_matrices = {}
        subject_files = list(files_dic[subject].keys())
        subject_files = sorted(subject_files,key=sort_key)
        

        for tr, file in enumerate(subject_files):
            print(tr)
            print(file)
            path = files_dic[subject][file]
            filename = folder + '/' + file
            # print(path)

            #Import the data from a sample file.
            data = open_mat_file(path)

            #Get signal, filter and downsample.
            signal = data['agr_source_data']
            filtered_signal = sosfiltfilt(butt_filter1, signal, axis=0)
            filtered_signal = sosfiltfilt(butt_filter2, filtered_signal, axis=0)
            # downsampled_signal = filtered_signal[::10,:]
            # analytic_signal = hilbert(downsampled_signal,axis=0)
            # no downsampling
            analytic_signal = hilbert(filtered_signal,axis=0)

            sg = int(np.floor(100/(25))*2+1)
            ts1 = savgol_filter(np.real(analytic_signal),sg,1,axis = 0,mode = 'interp')
            ts2 = savgol_filter(np.imag(analytic_signal),sg,1,axis = 0,mode = 'interp')
            ts3 = ts1+1j*ts2
            
            # time_windows = segment_time_series(ts3, window_len, overlap_len, 200)
            # segment data with tap-locked windows
            tr_samples = samples[tr][:,subj]
            time_windows = segment_time_series_taplocked(ts3, window_len, 2000,tr_samples)
            
            corr_matrix = []
            for window in time_windows:
                corr_matrix.append(np.corrcoef(window, rowvar=False))
            corr_matrix = np.array(corr_matrix)
            corr_matrix = corr_matrix - np.mean(corr_matrix,axis=0)
            corr_matrices[filename] = corr_matrix
        print('correlation matrices calculated.')

        matrix,sizes = corr_matrix_stack(corr_matrices)
        dist_matrix = np.zeros((len(matrix),len(matrix)))
        for i,j in combinations(range(len(matrix)),2):
            print(i)
            matrix1 = matrix[i,:,:]
            matrix2 = matrix[j,:,:]
            dist_matrix[i,j] = corr_dist(matrix1,matrix2)
            dist_matrix[j,i] = dist_matrix[i,j]
        
        print('distance matrices calculated.')
        subject_matrices[subject] = {'distances': dist_matrix, 'sizes':sizes}
    
    # filename_save = '/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/Italo/correlation_distances/dyad_' + folder + '_distances.pkl'
    filename_save = '/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/taplocked/correlation_distances/dyad_' + folder + '_distances.pkl'
    print(filename_save)
    save_to_pickle(subject_matrices, filename_save)

2022100401
files listed.
subj1
subj1_tr_1.mat
subj1_tr_2.mat
subj1_tr_3.mat
subj1_tr_4.mat
subj1_tr_5.mat
subj1_tr_6.mat
subj1_tr_7.mat
subj1_tr_8.mat
subj1_tr_9.mat
subj1_tr_10.mat
subj1_tr_11.mat
subj1_tr_12.mat
subj2
subj2_tr_1.mat
subj2_tr_2.mat
subj2_tr_3.mat
subj2_tr_4.mat
subj2_tr_5.mat
subj2_tr_6.mat
subj2_tr_7.mat
subj2_tr_8.mat
subj2_tr_9.mat
subj2_tr_10.mat
subj2_tr_11.mat
subj2_tr_12.mat


In [17]:
path

'/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/2022100401/subj1_tr_6.mat'

In [ ]:
data = open_mat_file(path)

'tr12'

In [123]:
folders

['20220713',
 '20220721',
 '20220804',
 '20220808',
 '20220810',
 '20220811',
 '20220815',
 '20220816',
 '20221003',
 '2022100401',
 '2022100402',
 '20221005']

In [111]:
# compute and save errors
errors={}
for folder in folders:
    errors_folder={}   
    print(folder)
    files_dic = {}
#    for folder in folders:
#    folder_dic = {}
    directory_path = '/ssd/zhibin/1overf/Cleaned_sourcedata/cortical_source_data/' + folder
    files = list_files_in_directory(directory_path)

    # load event time points
    beha_filename = '/home/zhibinz2/Documents/GitHub/finger_tapping_behavioral_data/clean_' + str(folder) + '_bpchan.mat'
    beh_data =  loadmat(beha_filename)
    samples = beh_data['samples'][0]

    for filename in files:
        subject, rest = filename.split('_', 1)
        if subject not in files_dic:
            files_dic[subject] = {}
        file = folder + '/' + filename
        files_dic[subject][filename] = directory_path + '/' + filename
    print('files listed.')

    for subj,subject in enumerate(['subj1','subj2']):
        # print(subj)
        print(subject) 
        corr_matrices = {}
        subject_files = list(files_dic[subject].keys())
        subject_files = sorted(subject_files,key=sort_key) 

        for tr, file in enumerate(subject_files):
            # print(tr)
            print(file)
            path = files_dic[subject][file]
            filename = folder + '/' + file
            # print(path)

            # time_windows = segment_time_series(ts3, window_len, overlap_len, 200)
            # segment data with tap-locked windows
            tr_samples = samples[tr][:,subj]
            time_windows = segment_time_series_taplocked(ts3, window_len, 2000,tr_samples)

            # Compute difference(error) of the samples
            if subject == 'subj1':
                tr_samples_both = samples[tr]
                num_tp=len(tr_samples_both)
                tr_error=np.zeros(num_tp)
                for tp in range(num_tp):
                    tr_error[tp]=np.abs(tr_samples_both[tp][0]-tr_samples_both[tp][1])
                errors_folder[file.split('_')[1]+(file.split('_')[2]).split('.')[0]]=tr_error
    
    errors[folder]=errors_folder
                    
            

20220713
files listed.
subj1
subj1_tr_1.mat
subj1_tr_2.mat
subj1_tr_3.mat


/tmp/ipykernel_2168604/3717702527.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(segments)


subj1_tr_4.mat
subj1_tr_5.mat
subj1_tr_6.mat
subj1_tr_7.mat
subj1_tr_8.mat
subj1_tr_9.mat
subj1_tr_10.mat
subj1_tr_11.mat
subj1_tr_12.mat
subj2
subj2_tr_1.mat
subj2_tr_2.mat
subj2_tr_3.mat
subj2_tr_4.mat
subj2_tr_5.mat
subj2_tr_6.mat
subj2_tr_7.mat
subj2_tr_8.mat
subj2_tr_9.mat
subj2_tr_10.mat
subj2_tr_11.mat
subj2_tr_12.mat
20220721
files listed.
subj1
subj1_tr_1.mat
subj1_tr_2.mat
subj1_tr_3.mat
subj1_tr_4.mat
subj1_tr_5.mat
subj1_tr_6.mat
subj1_tr_7.mat
subj1_tr_8.mat
subj1_tr_9.mat
subj1_tr_10.mat
subj1_tr_11.mat
subj1_tr_12.mat
subj2
subj2_tr_1.mat
subj2_tr_2.mat
subj2_tr_3.mat
subj2_tr_4.mat
subj2_tr_5.mat
subj2_tr_6.mat
subj2_tr_7.mat
subj2_tr_8.mat
subj2_tr_9.mat
subj2_tr_10.mat
subj2_tr_11.mat
subj2_tr_12.mat
20220804
files listed.
subj1
subj1_tr_1.mat
subj1_tr_2.mat
subj1_tr_3.mat
subj1_tr_4.mat
subj1_tr_5.mat
subj1_tr_6.mat
subj1_tr_7.mat
subj1_tr_8.mat
subj1_tr_9.mat
subj1_tr_10.mat
subj1_tr_11.mat
subj1_tr_12.mat
subj2
subj2_tr_1.mat
subj2_tr_2.mat
subj2_tr_3.mat
subj2_tr_

In [112]:
subject_files

['subj2_tr_1.mat',
 'subj2_tr_2.mat',
 'subj2_tr_3.mat',
 'subj2_tr_4.mat',
 'subj2_tr_5.mat',
 'subj2_tr_6.mat',
 'subj2_tr_7.mat',
 'subj2_tr_8.mat',
 'subj2_tr_9.mat',
 'subj2_tr_10.mat',
 'subj2_tr_11.mat',
 'subj2_tr_12.mat']

In [113]:
errors_folder.keys()

dict_keys(['tr1', 'tr2', 'tr3', 'tr4', 'tr5', 'tr6', 'tr7', 'tr8', 'tr9', 'tr10', 'tr11', 'tr12'])

In [120]:
errors_folder['tr1']

array([652., 550., 328., 244.,  97., 661., 686., 570., 459., 436., 336.,
        96., 141., 516., 696., 644., 643., 210., 121., 473., 631., 664.,
       715., 383.,  81., 185., 738., 499., 335.,  36., 253., 416., 739.,
       482., 239., 364., 234., 345., 641., 597., 851., 666., 520., 492.,
       561., 497., 573., 533., 559., 554., 509., 524., 500., 535., 647.,
       707., 745., 720., 742., 814., 708., 615., 625., 463., 413., 852.,
       551., 436., 344., 417., 501., 403., 488., 389., 303., 445., 546.,
       513., 456., 473., 455., 452., 390., 325., 317., 358., 536., 425.,
       222., 364., 473., 461., 485., 547., 611., 501., 505., 515., 489.,
       525., 580., 494., 645., 781., 688., 680., 746., 800., 736., 748.,
       655., 652., 558., 716., 609., 676., 578., 555., 453., 466., 651.,
       614., 549., 494., 407., 681., 646., 677., 558., 644., 525., 581.,
       382., 275., 196., 150., 271., 494., 563., 323., 318., 169., 101.,
       355., 393., 326., 336., 470., 463., 434., 41

In [121]:
errors['20220713']['tr1']

array([237., 200.,  73.,  72.,  61.,  35., 175.,  87., 124.,  74., 102.,
       152., 205.,  75., 149.,  63.,  22.,  22.,  40.,  24.,  32.,  83.,
       113.,  74.,  27.,   0.,  43.,  23., 103.,  73.,  51.,  45.,  61.,
         3.,   8.,  23., 131., 108.,  87.,  13., 385., 353., 289., 244.,
       333., 123.,  73.,  63.,  72.,  22.,   3.,  19.,   6.,  37.,  31.,
        49.,  75.,  78.,  21.,  48.,  29.,  97., 109., 142.,  78., 321.,
       203., 233.,  22.,   2., 169., 148.,  75.,  61.,  40.,  37.,  24.,
        66.,  32.,  15.,  52.,  18., 134., 139., 131., 145.,  24.,  20.,
        10., 103.,   1., 124.,   8.,  83., 142.,  70.,  71.,   2.,  93.,
       141.,  53.,  43., 161., 185.,  48.,  60.,  95.,  69., 120., 212.,
         2.,   3.,  53.,  53., 150., 236., 249., 185., 241., 219., 217.,
       124.,  62., 233., 445., 211.,  94.,  28.,  34.,  64., 234., 194.,
       185., 201., 302., 323., 453., 473., 198., 153., 168.,   9.,  93.,
        11., 116.,  48., 216., 120., 245.,   8.,  3

In [127]:
from hdf5storage import loadmat, savemat 

In [124]:
folder

'20221005'

In [125]:
errors[folder]

{'tr1': array([652., 550., 328., 244.,  97., 661., 686., 570., 459., 436., 336.,
         96., 141., 516., 696., 644., 643., 210., 121., 473., 631., 664.,
        715., 383.,  81., 185., 738., 499., 335.,  36., 253., 416., 739.,
        482., 239., 364., 234., 345., 641., 597., 851., 666., 520., 492.,
        561., 497., 573., 533., 559., 554., 509., 524., 500., 535., 647.,
        707., 745., 720., 742., 814., 708., 615., 625., 463., 413., 852.,
        551., 436., 344., 417., 501., 403., 488., 389., 303., 445., 546.,
        513., 456., 473., 455., 452., 390., 325., 317., 358., 536., 425.,
        222., 364., 473., 461., 485., 547., 611., 501., 505., 515., 489.,
        525., 580., 494., 645., 781., 688., 680., 746., 800., 736., 748.,
        655., 652., 558., 716., 609., 676., 578., 555., 453., 466., 651.,
        614., 549., 494., 407., 681., 646., 677., 558., 644., 525., 581.,
        382., 275., 196., 150., 271., 494., 563., 323., 318., 169., 101.,
        355., 393., 326., 336.,

In [130]:
import hdf5storage

ses12_errors=[]

for folder in folders:
    print(folder)
    ses12_errors.append(errors[folder])

ses12_errors.append(folders)


20220713
20220721
20220804
20220808
20220810
20220811
20220815
20220816
20221003
2022100401
2022100402
20221005


In [ ]:

# Convert the list to an object array (equivalent to MATLAB cell array)
ses12_errors_cell_array = np.empty(len(ses12_errors), dtype=object)
# Fill the object array with the data
for idx, arr in enumerate(ses12_errors):
    ses12_errors_cell_array[idx] = arr
# Save the object array (cell-like structure) to a .mat file
hdf5storage.savemat('ses12_errors.mat', {'cell_data': ses12_errors_cell_array}, format='7.3')
    
# 30s

In [116]:
savemat('errors.mat',errors)


In [117]:
outdict=loadmat('errors.mat')

In [118]:
outdict['20220713']['tr1']

array([237., 200.,  73.,  72.,  61.,  35., 175.,  87., 124.,  74., 102.,
       152., 205.,  75., 149.,  63.,  22.,  22.,  40.,  24.,  32.,  83.,
       113.,  74.,  27.,   0.,  43.,  23., 103.,  73.,  51.,  45.,  61.,
         3.,   8.,  23., 131., 108.,  87.,  13., 385., 353., 289., 244.,
       333., 123.,  73.,  63.,  72.,  22.,   3.,  19.,   6.,  37.,  31.,
        49.,  75.,  78.,  21.,  48.,  29.,  97., 109., 142.,  78., 321.,
       203., 233.,  22.,   2., 169., 148.,  75.,  61.,  40.,  37.,  24.,
        66.,  32.,  15.,  52.,  18., 134., 139., 131., 145.,  24.,  20.,
        10., 103.,   1., 124.,   8.,  83., 142.,  70.,  71.,   2.,  93.,
       141.,  53.,  43., 161., 185.,  48.,  60.,  95.,  69., 120., 212.,
         2.,   3.,  53.,  53., 150., 236., 249., 185., 241., 219., 217.,
       124.,  62., 233., 445., 211.,  94.,  28.,  34.,  64., 234., 194.,
       185., 201., 302., 323., 453., 473., 198., 153., 168.,   9.,  93.,
        11., 116.,  48., 216., 120., 245.,   8.,  3